In [1]:
import os
os.chdir('/Users/ilyeshammouda/Desktop/Ilyes/2A ENSAE/stage/code_stage_2_A_ENSAE/Stage_2A_ENSAE/code') 
# We position ourselves in the directory where the main classes are located.
print(f" You are currently in the directory: {os.getcwd()}")

 You are currently in the directory: /Users/ilyeshammouda/Desktop/Ilyes/2A ENSAE/stage/code_stage_2_A_ENSAE/Stage_2A_ENSAE/code


In [2]:
from Zoro import *
from functions import *

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import copy

In [3]:
mpl.style.use('seaborn')

In [4]:
# problem set up
n = 2000
s = int(0.4*n)
noiseamp = 0.001 # noise amplitude
obj_func = SparseQuadric(n, s, noiseamp)

# Choose initialization
x0    = np.random.randn(n)
x0    = 100*x0/np.linalg.norm(x0)
xx0   = copy.deepcopy(x0)

sparsity = s
#sparsity = int(0.1*len(x0)) # This is a decent default, if no better estimate is known. 

# Parameters for ZORO. Defaults are fine in most cases
params = {"step_size":1.0, "delta": 0.0001, "max_cosamp_iter": 10, 
          "cosamp_tol": 0.5,"sparsity": sparsity,
          "num_samples": int(np.ceil(np.log(len(x0))*sparsity))}

performance_log_ZORO = [[0, obj_func(x0)]]

In [ ]:
#IHT set up 
threshold_IHT=2
step_IHT=0.0000001
itt_IHT=30 
C_IHT=0.9
lamda_IHT=0.1

In [5]:
opt_Co  = ZORO(x0, obj_func, params,algo='CoSaMP', function_budget= int(1e6))
opt_Lasso  = ZORO(x0, obj_func, params,algo='Lasso', function_budget= int(1e6))
opt_DLasso  = ZORO(x0, obj_func, params,algo='DLasso', function_budget= int(1e6))
opt_IHT  = ZORO(x0, obj_func, params,algo='IHT_Classique', function_budget= int(1e6),s=sparsity)
opt_IHT  = ZORO(x0, obj_func, params,algo='IHT_Classique', function_budget= int(1e6),s=sparsity,itt_IHT=itt_IHT)
opt_IHT_ad  = ZORO(x0, obj_func, params,algo='IHT_ad', function_budget= int(1e6),threshold_IHT=threshold_IHT,C_IHT=C_IHT,lamda_IHT=lamda_IHT,itt_IHT=itt_IHT)

# the actual optimization routine
termination=False

In [7]:
performance_log_ZORO_IHT=opt_IHT_ad.Zoro()

Estimated f(x_k): 3988.027890  function evals: 12162
Estimated f(x_k): 4739.060176  function evals: 24324
Estimated f(x_k): 7090.062745  function evals: 36486
Estimated f(x_k): 11041.006406  function evals: 48648
Estimated f(x_k): 16591.861972  function evals: 60810
Estimated f(x_k): 23742.600255  function evals: 72972
Estimated f(x_k): 32493.192059  function evals: 85134
Estimated f(x_k): 42843.608200  function evals: 97296
Estimated f(x_k): 54793.819487  function evals: 109458
Estimated f(x_k): 68343.796737  function evals: 121620
Estimated f(x_k): 83493.510761  function evals: 133782
Estimated f(x_k): 100242.932354  function evals: 145944
Estimated f(x_k): 118592.032358  function evals: 158106
Estimated f(x_k): 138540.781568  function evals: 170268
Estimated f(x_k): 160089.150823  function evals: 182430
Estimated f(x_k): 183237.110927  function evals: 194592
Estimated f(x_k): 207984.632678  function evals: 206754
Estimated f(x_k): 234331.686904  function evals: 218916
Estimated f(x_

In [6]:
performance_log_ZORO_IHT=opt_IHT.Zoro()

Estimated f(x_k): 3988.027890  function evals: 12162
Estimated f(x_k): 2989.634778  function evals: 24324
Estimated f(x_k): 2533.970031  function evals: 36486
Estimated f(x_k): 2370.666278  function evals: 48648
Estimated f(x_k): 2383.445589  function evals: 60810
Estimated f(x_k): 2468.132621  function evals: 72972
Estimated f(x_k): 2555.927606  function evals: 85134
Estimated f(x_k): 2617.007948  function evals: 97296
Estimated f(x_k): 2640.379995  function evals: 109458
Estimated f(x_k): 2659.895167  function evals: 121620
Estimated f(x_k): 2677.010468  function evals: 133782
Estimated f(x_k): 2679.273441  function evals: 145944
Estimated f(x_k): 2686.930142  function evals: 158106
Estimated f(x_k): 2690.414871  function evals: 170268
Estimated f(x_k): 2690.414871  function evals: 182430
Estimated f(x_k): 2690.414870  function evals: 194592
Estimated f(x_k): 2690.414871  function evals: 206754
Estimated f(x_k): 2690.414871  function evals: 218916
Estimated f(x_k): 2690.414871  funct

In [ ]:
performance_log_ZORO_DLasso=opt_DLasso.Zoro()

In [ ]:
performance_log_ZORO_Co=opt_Co.Zoro()
performance_log_ZORO_Lasso=opt_Lasso.Zoro()
performance_log_ZORO_DLasso=opt_DLasso.Zoro()

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.array(performance_log_ZORO_Co)[:,0],
 np.log10(np.array(performance_log_ZORO_Co)[:,1]), linewidth=1, label = "ZORO_original")
ax.plot(np.array(performance_log_ZORO_Lasso)[:,0],
 np.log10(np.array(performance_log_ZORO_Lasso)[:,1]), linewidth=1, label = "ZORO_Lasso")
#ax.plot(np.array(performance_log_ZORO_DLasso)[:,0],
 #np.log10(np.array(performance_log_ZORO_DLasso)[:,1]), linewidth=1, label = "ZORO_DLasso")
plt.xlabel('number of queries')
plt.ylabel('$log($f(x)$)$')
leg = ax.legend()
plt.show()